Install dependencies

In [ ]:
!pip3 install -r requirements.txt -U
!python3 -m spacy download en_core_web_md

Suppress warnings

In [1]:
import warnings
warnings.filterwarnings('ignore')

Import core lib

In [2]:
import sys
sys.path.append('./core')

Import dependencies

In [3]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import re
import os
import tf_metrics
from imblearn import over_sampling, under_sampling, combine
from collections import Counter
from tensorflow.core.util import event_pb2
from tensorflow.python.lib.io import tf_record
from misc.doc_reader import get_document_list
import matplotlib
import matplotlib.pyplot as plt
#import random
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import sklearn.preprocessing as preprocessing
from models.role_pattern_extractor import RolePatternExtractor

Using TensorFlow backend.


Suppress tensorflow warnings

In [4]:
tf.get_logger().setLevel('ERROR')

Define constants

In [5]:
BATCH_SIZE = 2
N_SPLITS = 5
USE_TEST_SET = True
USE_PATTERN_EMBEDDING = True

ZERO_CLASS = 'none'
LABELS_TO_EXCLUDE = [
	#'cites',
	'cites_as_review',
	#'extends', 
	#'uses_data_from', 
	#'uses_method_in',
]
OVERSAMPLE = False
UNDERSAMPLE = False

TRAIN_EPOCHS = None
EVALUATION_SECONDS = 5
MAX_STEPS = (10**5)/2
EVALUATION_STEPS = MAX_STEPS/10
MODEL_DIR = './model'
TF_MODEL = 'USE_MLQA'
MODEL_MANAGER = RolePatternExtractor({'tf_model':TF_MODEL})

Loading spacy model <en_core_web_md>...


Define function for converting input datasets from csv to pandas dataframe

In [6]:
def get_dataframe(dataset_file):
	# Load dataset
	df = pd.read_csv(dataset_file, sep='	')
	#print(df.dtypes)

	# Get target values list
	df['citfunc'].replace(np.NaN, 'none', inplace=True)
	df['citfunc'] = df['citfunc'].map(lambda x: x.strip())
	# Remove rows with excluded labels
	for label in LABELS_TO_EXCLUDE:
		df.loc[df.citfunc == label, 'citfunc'] = ZERO_CLASS
	# Remove bad rows
	df['citfunc'].replace('ERROR', 'none', inplace=True)
	df = df[df.citfunc != 'none']
	# Extract target list
	target_list = df.pop('citfunc').values.tolist()

	# Extract features from dataframe
	df = df[['anchorsent','sectype']]
	
	# Remove null values
	df['anchorsent'].replace(np.NaN, '', inplace=True)
	df['sectype'].replace(np.NaN, 'none', inplace=True)

	df = df[df.anchorsent != '']
	df['anchorsent'] = df['anchorsent'].map(lambda x: re.sub(r'\[\[.*\]\]','',x))
	df['anchorsent'] = df['anchorsent'].map(lambda x: re.sub(r'[^\x00-\x7F]+',' ',x))

	if USE_PATTERN_EMBEDDING:
		extra_list = []
		for text in df['anchorsent'].values:
			extra = list(Counter(pattern['predicate'] for pattern in MODEL_MANAGER.get_role_pattern_list(text)).keys())
			extra_list.append(extra[0] if len(extra)>0 else '')
		df['main_predicate'] = extra_list
	
	# Print dataframe
	print('Dataframe')
	print(df)
	
	# Return dataset
	feature_list = df.columns.values.tolist()
	x_dict = {feature: df[feature].tolist() for feature in feature_list}
	y_list = target_list
	return {'x':x_dict, 'y':y_list}

Define function for casting dataset to numpy arrays

In [7]:
def numpyfy_dataset(set):
	set['x'] = {k: np.array(v) for k,v in set['x'].items()}
	set['y'] = np.array(set['y'])

Define function for encoding a dataset, from string to numerical representations

In [8]:
def encode_dataset(dataset):
	# Embed anchor sentences into vectors
	for key,value in dataset.items():
		df = value['x']
		# Embed anchor sentences
		cache_file = f'{TF_MODEL}.{key}.anchorsent.embedding_cache.pkl'
		if os.path.isfile(cache_file):
			with open(cache_file, 'rb') as f:
				embedded_sentences = pickle.load(f)
		else:
			df['anchorsent'] = list(df['anchorsent'])
			embedded_sentences = MODEL_MANAGER.embed(df['anchorsent'])
			with open(cache_file, 'wb') as f:
				pickle.dump(embedded_sentences, f)
		df['anchorsent'] = embedded_sentences
		# Embed extra info
		if USE_PATTERN_EMBEDDING:
			cache_file = f'{TF_MODEL}.{key}.extra.embedding_cache.pkl'
			if os.path.isfile(cache_file):
				with open(cache_file, 'rb') as f:
					embedded_extra = pickle.load(f)
			else:
				df['main_predicate'] = list(df['main_predicate'])
				embedded_extra = MODEL_MANAGER.embed(df['main_predicate'])
				with open(cache_file, 'wb') as f:
					pickle.dump(embedded_extra, f)
			df['main_predicate'] = embedded_extra

	# Encode labels
	label_encoder_target = LabelEncoder()
	label_encoder_target.fit([e for set in dataset.values() for e in set['y']])
	print('Label classes:', list(label_encoder_target.classes_))
	for set in dataset.values():
		set['y'] = label_encoder_target.transform(set['y'])

	# Encode sectypes
	all_sectypes = [e for set in dataset.values() for e in set['x']['sectype']]
	label_encoder_sectype = LabelEncoder()
	all_sectypes = label_encoder_sectype.fit_transform(all_sectypes)
	onehot_encoder_sectype = OneHotEncoder()
	onehot_encoder_sectype.fit(all_sectypes.reshape(-1, 1))
	print('Sectype classes:', list(label_encoder_sectype.classes_))
	for set in dataset.values():
		labeled_sectypes = label_encoder_sectype.transform(set['x']['sectype'])
		set['x']['sectype'] = onehot_encoder_sectype.transform(labeled_sectypes.reshape(-1, 1)).toarray()[:,1:]

	# Input features to numpy array
	for set in dataset.values():
		numpyfy_dataset(set)
	# Return number of target classes
	return len(label_encoder_target.classes_)

Define function for resampling the dataset

In [9]:
def resample_dataset(set, oversample=True, undersample=True):
	#numpyfy_dataset(set)
	print('Dataset size before re-sampling:', len(set['y']))

	# Build combined features
	combined_features_sizes = {}
	combined_features_list = []
	for feature in zip(*set['x'].values()):
		combined_features = []
		for e,data in enumerate(feature):
			if type(data) in [np.ndarray,list,tuple]:
				data_list = list(data)
				combined_features.extend(data_list)
				combined_features_sizes[e] = (len(data_list), type(data[0]))
			else:
				combined_features.append(data)
				combined_features_sizes[e] = (1, type(data))
		combined_features_list.append(combined_features)
	#print(combined_features_list[0])

	# Oversample data
	combined_features_list = np.array(combined_features_list, dtype=np.object)
	#combined_features_list, set['y'] = over_sampling.RandomOverSampler(sampling_strategy='all').fit_sample(combined_features_list, set['y'])
	if oversample and undersample:
		combined_features_list, set['y'] = combine.SMOTETomek().fit_sample(combined_features_list, set['y'])
	elif oversample:
		combined_features_list, set['y'] = over_sampling.ADASYN().fit_sample(combined_features_list, set['y'])
	elif undersample:
		combined_features_list, set['y'] = under_sampling.NeighbourhoodCleaningRule().fit_sample(combined_features_list, set['y'])

	# Separate features
	new_combined_features_list = []
	for combined_features in combined_features_list:
		new_combined_features = []
		start = 0
		for e,(size,dtype) in combined_features_sizes.items():
			feature = combined_features[start:start+size]
			if size > 1:
				#feature = np.array(feature, dtype=dtype)
				feature = np.array(feature, dtype=np.float32)
			else:
				feature = feature[0]
			new_combined_features.append(feature)
			start += size
		new_combined_features_list.append(new_combined_features)
	#print(new_combined_features_list[0])
	separated_features = list(zip(*new_combined_features_list))

	for feature, value in zip(set['x'].keys(), separated_features):
		set['x'][feature] = value
	print('Dataset size after re-sampling:', len(set['y']))
	numpyfy_dataset(set)

Define function for getting the dataframe feature shapes

In [10]:
def get_dataframe_feature_shape(df, feature):
	first_element = df[feature][0]
	shape = first_element.shape if type(first_element) is np.ndarray else ()
	return tf.feature_column.numeric_column(feature, shape=shape)

Define function to convert a data-set into a data-list

In [11]:
def listify_dataset(dataset):
	dataset_xs = zip(*dataset['x'].values())
	dataset_xs = map(lambda x: tuple((k,v) for k,v in zip(dataset['x'].keys(),x)), dataset_xs)
	return list(zip(dataset_xs, dataset['y']))

Define function to convert a data-set into a data-list

In [12]:
def dictify_datalist(datalist):
	xs, y = zip(*datalist)
	y_list = np.array(y)
	xs = zip(*xs)
	xs_dict = {}
	for x_tuples in xs:
		feature_names, x_tuples = zip(*x_tuples)
		feature = feature_names[0]
		xs_dict[feature] = np.array(x_tuples)
		#print(feature, len(xs_dict[feature]))
	#print('y', len(y_list))
	return {
		'x': xs_dict,
		'y': y_list
	}

Define the DNN classifier model

In [ ]:
def build_model_fn(feature_columns, n_classes):
	def model_fn(
		features, # This is batch_features from input_fn
		labels,   # This is batch_labels from input_fn
		mode):	# And instance of tf.estimator.ModeKeys, see below

		if mode == tf.estimator.ModeKeys.PREDICT:
			tf.logging.info("my_model_fn: PREDICT, {}".format(mode))
		elif mode == tf.estimator.ModeKeys.EVAL:
			tf.logging.info("my_model_fn: EVAL, {}".format(mode))
		elif mode == tf.estimator.ModeKeys.TRAIN:
			tf.logging.info("my_model_fn: TRAIN, {}".format(mode))

		# Create the layer of input
		input_layer = tf.feature_column.input_layer(features, feature_columns)
		#input_layer = tf.expand_dims(input_layer, 1)

		input_layer = tf.layers.Dense(16, #3, padding='same',
			activation=tf.nn.tanh, 
			#kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.003)
		)(input_layer)

		input_layer = tf.layers.Dropout()(input_layer)
		#input_layer = tf.layers.Flatten()(input_layer)

		logits = tf.layers.Dense(n_classes, 
			#kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.003)
		)(input_layer)

		# class_ids will be the model prediction for the class (Iris flower type)
		# The output node with the highest value is our prediction
		def sample(logits, random=True):
			if random:
				u = tf.random_uniform(tf.shape(logits), dtype=logits.dtype)
				logits -= tf.log(-tf.log(u))
			return tf.argmax(logits, axis=1)

		predictions = { 'class_ids': sample(logits, random=False) }

		# 1. Prediction mode
		# Return our prediction
		if mode == tf.estimator.ModeKeys.PREDICT:
			return tf.estimator.EstimatorSpec(mode, predictions=predictions)

		# Evaluation and Training mode

		# Calculate the loss
		loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
		loss += tf.losses.get_regularization_loss()

		# Calculate the accuracy between the true labels, and our predictions
		y_true=labels
		y_pred=predictions['class_ids']
		average_type_list = ['micro','macro','weighted']
		metrics = {}
		for average in average_type_list:
			metrics[f'precision_{average}'] = tf_metrics.precision(y_true, y_pred, n_classes, average=average)
			metrics[f'recall_{average}'] = tf_metrics.recall(y_true, y_pred, n_classes, average=average)
			metrics[f'f1_{average}'] = tf_metrics.f1(y_true, y_pred, n_classes, average=average)

		# 2. Evaluation mode
		# Return our loss (which is used to evaluate our model)
		# Set the TensorBoard scalar my_accurace to the accuracy
		# Obs: This function only sets value during mode == ModeKeys.EVAL
		# To set values during training, see tf.summary.scalar
		if mode == tf.estimator.ModeKeys.EVAL:
			return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

		# If mode is not PREDICT nor EVAL, then we must be in TRAIN
		assert mode == tf.estimator.ModeKeys.TRAIN, "TRAIN is only ModeKey left"

		# 3. Training mode

		# Default optimizer for DNNClassifier: Adagrad with learning rate=0.05
		# Our objective (train_op) is to minimize loss
		# Provide global step counter (used to count gradient updates)
		#optimizer = tf.train.AdagradOptimizer(0.05)
		#optimizer = tf.train.AdamOptimizer()
		optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=0.01, l2_regularization_strength=0.003)
		train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

		# For Tensorboard
		for metric_name, metric in metrics.items():
			tf.summary.scalar(metric_name, metric[1])

		# Return training operations: loss and train_op
		return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
	return model_fn

Define function for extracting summaries (statistics) from tensorboard events

In [ ]:
def get_summary_results(summary_dir):
	def my_summary_iterator(path):
		for r in tf_record.tf_record_iterator(path):
			yield event_pb2.Event.FromString(r)

	result_list = []
	document_list = get_document_list(summary_dir)
	#print(document_list)
	for filename in document_list:
		print(filename)
		if not os.path.basename(filename).startswith('events.'):
			continue
		value_dict = {}
		for event in my_summary_iterator(filename):
			for value in event.summary.value:
				tag = value.tag
				if tag not in value_dict:
					value_dict[tag]=[]
				value_dict[tag].append((event.step, value.simple_value))
		result_list.append({'event_name':filename, 'results':value_dict})
	return result_list

Define function for training and evaluating

In [ ]:
def train_and_evaluate(trainset, testset, num_epochs, batch_size, max_steps, model_dir, feature_columns, n_classes):
	# Create a custom estimator using model_fn to define the model
	tf.logging.info("Before classifier construction")
	run_config = tf.estimator.RunConfig(
		model_dir=model_dir,
		#save_checkpoints_secs=EVALUATION_SECONDS, 
		save_checkpoints_steps=EVALUATION_STEPS,
		#keep_checkpoint_max=3,
	)
	estimator = tf.estimator.Estimator(
		model_fn=build_model_fn(feature_columns, n_classes),
		config=run_config,
	)
	tf.logging.info("...done constructing classifier")

	# Build train input callback
	train_input_fn = tf.estimator.inputs.numpy_input_fn(
		x=trainset['x'],
		y=trainset['y'],
		num_epochs=num_epochs,
		batch_size=batch_size,
		shuffle=True
	)
	# Build test input callback
	test_input_fn = tf.estimator.inputs.numpy_input_fn(
		x=testset['x'],
		y=testset['y'],
		num_epochs=1,
		batch_size=batch_size,
		shuffle=False
	)

	train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=max_steps)
	eval_spec = tf.estimator.EvalSpec(input_fn=test_input_fn, throttle_secs=EVALUATION_SECONDS)

	tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Load dataset 1

In [ ]:
trainset = get_dataframe('training_all.csv')

Dataframe
                                             anchorsent       sectype  \
0     'In summary, the open nature of the Internet a...  introduction   
1     'Finally, a common data transformation method ...          none   
3     'Hucaljuk and Rakipovic [15] included an exper...       results   
4     'The first step in the modelling process is to...       results   
5     'Hucaljuk and Rakipovic [15] used a separate e...       results   
...                                                 ...           ...   
1552  'To support feature computation and combinatio...          none   
1553  'To query the lexical databases we use the ope...          data   
1554  'To compute the score, we use a series of feat...          data   
1555  'Using the strategy described in Section 3.3, ...          none   
1556  'It is computed by comparing all patents in th...          none   

     main_predicate  
0            create  
1         is called  
3          included  
4      will be used  
5  

Load dataset 2

In [ ]:
testset = get_dataframe('test_groundtruth_all.csv')

Dataframe
                                            anchorsent       sectype  \
0    'This is common in software design when the UI...  introduction   
1    'The most related items intersect first, and t...          none   
2    'In this study, participants performed a hiera...          none   
3    'Card sorting software, xSort (Arroz, 2008) wa...          none   
4    'The concept of using PA to analyze SD evaluat...  introduction   
..                                                 ...           ...   
295  'Data words are commonly studied in XML litera...          none   
296  'It was shown in [7] that the language L={(ad1...          none   
297  'Originally they were defined on words over in...          none   
298  'What all of these languages (with the sole ex...  introduction   
299  'Note that besides the operators in Definition...          none   

    main_predicate  
0      is assigned  
1                   
2        performed  
3         was used  
4     was proposed  

Encode dataset

In [ ]:
n_classes = encode_dataset({'train':trainset, 'test':testset})

Label classes: ['cites', 'extends', 'uses_data_from', 'uses_method_in']
Sectype classes: ['acknowledgements', 'background', 'conclusion', 'data', 'discussion', 'introduction', 'materials', 'methods', 'model', 'motivation', 'none', 'related work', 'results', 'scenario']


Get feature columns

In [ ]:
feature_columns = [get_dataframe_feature_shape(trainset['x'],feature) for feature in trainset['x'].keys()]

Merge dataset 1 and 2, because they have different distributions and thus we have to build new train and test sets. Before mergin we convert the datasets into datalists, this way we can easily shuffle them.

In [ ]:
trainlist = listify_dataset(trainset)
if USE_TEST_SET:
	testlist = listify_dataset(testset)
	datalist = trainlist + testlist
else:
	datalist = trainlist

Define function for plotting summary results

In [ ]:
def plot_summary_results(summary_results):
	plt.clf()
	plt_height = len(summary_results)
	_, axes = plt.subplots(nrows=plt_height, sharex=True, figsize=(14,15*plt_height))
	for e, (stat, value_list) in enumerate(summary_results.items()):
		ax = axes[e]
		#ax.set_ylim([0, 1])
		#ax.set_yticks(value_list)
		step_list,value_list=zip(*value_list)
		ax.plot(step_list, value_list)
		ax.set(xlabel='step', ylabel=stat)
		ax.grid()
	plt.show()

Define function for cross-validating the model

In [ ]:
def build_cross_validate_model(datalist):
	def get_best_stat_dict(summary_results_list):
		best_stat_dict = {}
		for summary_results in summary_results_list:
			for stat, value_list in summary_results.items():
				_,value_list=zip(*value_list)
				if not re.search(r'(f1|precision|recall)', stat):
					continue
				if stat not in best_stat_dict:
					best_stat_dict[stat] = []
				best_stat_dict[stat].append(max(value_list))
		return best_stat_dict

	def cross_validate_model(config, reporter):
		summary_results_list = []
		cross_validation = KFold(n_splits=config["N_SPLITS"], shuffle=True, random_state=1)
		for e, (train_index, test_index) in enumerate(cross_validation.split(datalist)):
			#print(f'-------- Fold {e} --------')
			#print(f'Train-set {e} indexes {train_index}')
			#print(f'Test-set {e} indexes {test_index}')
			# Split training and test set
			trainlist = [datalist[u] for u in train_index]
			trainset = dictify_datalist(trainlist)
			# Oversample training set (after sentences embedding)
			if config["OVERSAMPLE"] or config["UNDERSAMPLE"]:
				resample_dataset(trainset, oversample=config["OVERSAMPLE"], undersample=config["UNDERSAMPLE"])
			#print(f'Train-set {e} distribution', Counter(trainset['y']))
			testlist = [datalist[u] for u in test_index]
			testset = dictify_datalist(testlist)
			#print(f'Test-set {e} distribution', Counter(testset['y']))

			config_str = '_'.join(f'{key}={value}' for key,value in config.items())
			model_dir = f'{MODEL_DIR}{e}-{config_str}'
			train_and_evaluate(
				trainset=trainset, 
				testset=testset, 
				num_epochs=TRAIN_EPOCHS, 
				batch_size=config["BATCH_SIZE"], 
				max_steps=MAX_STEPS, 
				model_dir=model_dir, 
				feature_columns=feature_columns, 
				n_classes=n_classes
			)
			summary_results = get_summary_results(f'./{model_dir}/eval')
			summary_results = summary_results[-1]['results']
			summary_results_list.append(summary_results)
			#print(f'Test-set {e} results:', summary_results)
			best_stat_dict = get_best_stat_dict(summary_results_list)
			reporter(timesteps_total=e, reward=best_stat_dict["f1_macro"])
	return cross_validate_model

Perform automatic hyper-parameters tuning

In [ ]:
import ray
import ray.tune as tune
from ray.tune.schedulers import HyperBandScheduler

ray.init(num_cpus=4)
tune.run(build_cross_validate_model(datalist),
    name="my_experiment",
    config={ 
        "N_SPLITS": tune.grid_search([3, 4, 5, 6, 7, 8, 9, 10]), 
        "OVERSAMPLE": tune.grid_search([True, False]),
        "UNDERSAMPLE": tune.grid_search([True, False]),
        "BATCH_SIZE": tune.grid_search([1, 2, 3, 4, 5, 6, 7, 8]),
    },
    scheduler=HyperBandScheduler(reward_attr="reward"))

2019-10-22 21:13:51,058	INFO resource_spec.py:205 -- Starting Ray with 1.61 GiB memory available for workers and up to 0.82 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2019-10-22 21:13:51,522	WARNING hyperband.py:96 -- `reward_attr` is deprecated and will be removed in a future version of Tune. Setting `metric=reward` and `mode=max`.


== Status ==
Using HyperBand: num_stopped=0 total_brackets=0
Round #0:
Resources requested: 0/4 CPUs, 0/0 GPUs, 0.0/1.61 GiB heap, 0.0/0.54 GiB objects
Memory usage on this node: 5.3/8.0 GiB

== Status ==
Using HyperBand: num_stopped=0 total_brackets=10
Round #0:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {PENDING: 4, RUNNING: 1} 
  Bracket(Max Size (n)=8, Milestone (r)=27, completed=0.0%): {PENDING: 8} 
  Bracket(Max Size (n)=15, Milestone (r)=9, completed=0.0%): {PENDING: 15} 
  Bracket(Max Size (n)=34, Milestone (r)=3, completed=0.0%): {PENDING: 34} 
  Bracket(Max Size (n)=81, Milestone (r)=1, completed=0.0%): {PENDING: 81} 
Round #1:
  Bracket(Max Size (n)=5, Milestone (r)=81, completed=0.0%): {PENDING: 5} 
  Bracket(Max Size (n)=8, Milestone (r)=27, completed=0.0%): {PENDING: 8} 
  Bracket(Max Size (n)=15, Milestone (r)=9, completed=0.0%): {PENDING: 15} 
  Bracket(Max Size (n)=34, Milestone (r)=3, completed=0.0%): {PENDING: 34} 
  Bracket(Max Size (n)=81, Milesto

(pid=5311) /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=5311)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=5311) /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=5311)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=5311) /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type

(pid=5309) Dataset size before re-sampling: 1110
(pid=5312) Dataset size before re-sampling: 1110
(pid=5311) /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=5311)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=5311) /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=5311)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=5311) /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1ty

(pid=5312) W1022 21:14:21.621958 123145510055936 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
(pid=5312) Instructions for updating:
(pid=5312) Call initializer instance with the dtype argument instead of passing it to the constructor
(pid=5310) Dataset size after re-sampling: 2722
(pid=5310) WARNING: Logging before flag parsing goes to stderr.
(pid=5310) W1022 21:14:22.072638 123145488019456 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=5310) Instructions for updating:
(pid=5310) Use Variable.read_value. Variables in 2.X are in

(pid=5310) W1022 21:14:22.891736 123145488019456 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
(pid=5310) Instructions for updating:
(pid=5310) Use tf.where in 2.0, which has the same broadcast rule as np.where
(pid=5310) W1022 21:14:23.004464 123145488019456 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tf_metrics/__init__.py:152: The name tf.diag_part is deprecated. Please use tf.linalg.tensor_diag_part instead.
(pid=5310) 
(pid=5310) W1022 21:14:23.014734 123145488019456 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tf_metrics/__init__.py:140: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a f

(pid=5311) W1022 21:14:40.905765 123145476014080 basic_session_run_hooks.py:724] It seems that global step (tf.train.get_global_step) has not been increased. Current value (could be stable): 3942 vs previous value: 3942. You could increase the global step by passing tf.train.get_global_step() to Optimizer.apply_gradients or Optimizer.minimize.
(pid=5309) W1022 21:14:46.168151 123145569439744 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
(pid=5309) Instructions for updating:
(pid=5309) Use standard file APIs to check for files with this prefix.
(pid=5312) W1022 21:14:46.335248 123145510055936 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tens